In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import Sequential,MaxPool2d,Linear,ReLU
from functions_pytorch import *
from pytorch_util import *
from pytorch_models import ConvBatchRelu
from albumentations_helper import create_transform
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
import pickle
import numpy as np
import pretrainedmodels
from functools import partial

In [2]:
## configs ##
color = True
#shapes = (3,224,224)
HalfBatch = 8
outDim = 500
distanceFun = l2_distance
distanceFun_np = l2_distance_np
TTASize = 4
pct = 0.1

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
# combine whales with one image with new whale
newWhale_train2 = newWhale_train + [i[0] for i in Ids_train.loc[Ids_train.length2 == 1].Imgs.tolist()]
Ids_train2 = Ids_train.loc[Ids_train.length2>1]

Set up dataloader

In [5]:
aug = Compose([RandomContrast(p=0.5),RandomBrightness(p=0.5),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=25,scale_limit=0.05,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

In [6]:
gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch,True,num_workers=3,drop_last=True,worker_init_fn=worker_init_fn)
valid_dl = DataLoader(gen_val,HalfBatch,False,num_workers=3,drop_last=True,worker_init_fn=worker_init_fn)

Set up training

In [7]:
# convNet = Sequential(ConvBatchRelu(3,8,5,stride=2),\
#                      ConvBatchRelu(8,16,5,stride=2),\
#                      ConvBatchRelu(16,32,5,stride=2),\
#                      MaxPool2d(2))

# fcNet = Sequential(Linear(4608,32),ReLU(True),Linear(32,6))

# with torch.no_grad():
#     fcNet[2].weight.data.zero_()
#     fcNet[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

# stn = SpatialTransformerNet(convNet,fcNet)

In [8]:
#base = pretrainedmodels.resnet50()
# base = pretrainedmodels.densenet121()
# base = fine_tune_pretrainedmodels(base, outDim)
# model = Sequential(stn,base).to('cuda:0')

In [9]:
model = pretrainedmodels.densenet121()
model = fine_tune_pretrainedmodels(model, outDim).to('cuda:0')

In [10]:
loss_func = loss_func_generator_softmax(HalfBatch,distanceFun)

In [11]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [12]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:1.378513602774, val_loss:0.6938920617103577
epoch:1, train_loss:1.0786016539472048, val_loss:0.5873348116874695
epoch:2, train_loss:0.9780504273993721, val_loss:0.5312209129333496
epoch:3, train_loss:0.9467551719621231, val_loss:0.5066378712654114
epoch:4, train_loss:0.9106383736097747, val_loss:0.4823993742465973
Training completed in 100.12278580665588s


Fine-tune previous layers

In [ ]:
# for ResNet50
# set_requires_grad([model[1].layer4,model[1].layer3,model[1].layer2],True)
# opt = Adam([
#             {"params": model[1].layer4.parameters(), "lr": 1e-5},
#             {"params": model[1].layer3.parameters(), "lr": 1e-6},
#             {"params": model[1].layer2.parameters(), "lr": 5e-7},
#             {"params": model[1].last_linear.parameters(), "lr": 2e-5},
#             {"params": model[0].parameters(), "lr": 2e-5}
#             ])

In [13]:
# for DenseNet121
name_list = ['denseblock3','transition3','denseblock4','norm5']
set_requires_grad_paraList(gather_parameter_nameList(name_list,model),True)

In [14]:
opt = Adam([
            {"params": gather_parameter_nameList(['denseblock4','norm5'],model), "lr": 1e-4},
            {"params": gather_parameter_nameList(['denseblock3','transition3'],model), "lr": 1e-5},
            {"params": gather_parameter_nameList(['denseblock2','transition2'],model), "lr": 1e-6},
            {"params": model.last_linear.parameters(), "lr": 2e-4}
            ])

In [15]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.7538976313067915, val_loss:0.36819979548454285
epoch:1, train_loss:0.5972001439194536, val_loss:0.2910773456096649
epoch:2, train_loss:0.44702712851551063, val_loss:0.3043462038040161
epoch:3, train_loss:0.39378237675447936, val_loss:0.2815174162387848
epoch:4, train_loss:0.3634778571320199, val_loss:0.22795656323432922
Training completed in 163.14613795280457s


In [16]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk1_softmax')

In [53]:
# parameters
k = 50
numBatch = 200
batchSize = 16
maxImg = Ids_train2.shape[0]-1

In [27]:
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)
aggFun = partial(np.quantile,q=pct,axis=(1,2))

In [28]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.25083487940630794

In [34]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [39]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [40]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.5731608559850787
epoch:1, train_loss:1.388969459168898
epoch:2, train_loss:1.2489690171564862
epoch:3, train_loss:1.1380954024895944
epoch:4, train_loss:1.1011318016410525
Training completed in 111.29576396942139s


In [41]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk2_softmax')

In [42]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.38537414965986394

In [43]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [56]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [57]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.9601198949449049
epoch:1, train_loss:0.9712737682250028
epoch:2, train_loss:0.880276007974734
epoch:3, train_loss:0.8759040593123827
epoch:4, train_loss:0.8341019145859395
Training completed in 110.96813440322876s


In [58]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk3_softmax')

In [59]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.4405689548546691

In [60]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [61]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [62]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.8243550097193223
epoch:1, train_loss:0.8108655268865856
epoch:2, train_loss:0.7433415876060235
epoch:3, train_loss:0.7531034713560115
epoch:4, train_loss:0.6718281174048049
Training completed in 110.70977807044983s


In [63]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk4_softmax')

In [64]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.5098484848484849

In [65]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [66]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [67]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.7204154378485158
epoch:1, train_loss:0.7026290195317216
epoch:2, train_loss:0.6978385180560618
epoch:3, train_loss:0.6413869630810016
epoch:4, train_loss:0.6409262851532039
Training completed in 110.86532378196716s


In [68]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk5_softmax')

In [69]:
k = 30

In [70]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.5361471861471861

In [71]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [72]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [73]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.7350096482722486
epoch:1, train_loss:0.6582221756053102
epoch:2, train_loss:0.6930301607957955
epoch:3, train_loss:0.6240561480730609
epoch:4, train_loss:0.6301098224895249
Training completed in 111.04898929595947s


In [74]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk5_softmax')

In [75]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.5807050092764379

In [76]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [77]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [78]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.6281781810554651
epoch:1, train_loss:0.5996673433873497
epoch:2, train_loss:0.5876219293784574
epoch:3, train_loss:0.5490044508882559
epoch:4, train_loss:0.5633841182201937
Training completed in 110.80002808570862s


In [79]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk6_softmax')

In [80]:
k=20

In [81]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.5880488559059988

In [82]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [83]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [84]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.6867647228833756
epoch:1, train_loss:0.640121043013419
epoch:2, train_loss:0.6268996229507232
epoch:3, train_loss:0.5960648732432903
epoch:4, train_loss:0.6031294748431346
Training completed in 111.36775135993958s


In [85]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk7_softmax')

In [86]:
k=20

In [87]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6148886827458256

In [88]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [89]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [90]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.6069116929922599
epoch:1, train_loss:0.616407717104818
epoch:2, train_loss:0.5550617157517235
epoch:3, train_loss:0.5776728700663223
epoch:4, train_loss:0.5586168837677585
Training completed in 111.10607242584229s


In [91]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk8_softmax')

In [92]:
k=10

In [93]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6322201607915894

In [94]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [95]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [96]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.8013952577618954
epoch:1, train_loss:0.7213272161252512
epoch:2, train_loss:0.7221795765102886
epoch:3, train_loss:0.6801592710076786
epoch:4, train_loss:0.6830412920747624
Training completed in 111.30374383926392s


In [97]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk8_softmax')

In [98]:
k=10

In [99]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6578695114409401

In [100]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [101]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [102]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.7194353253707859
epoch:1, train_loss:0.7132565244380894
epoch:2, train_loss:0.6710567396398777
epoch:3, train_loss:0.708072390753389
epoch:4, train_loss:0.6433616190171633
Training completed in 111.12934875488281s


In [103]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk8_softmax')

In [104]:
k=10

In [105]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6651205936920223

In [106]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [107]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [108]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:0.7180752332011858
epoch:1, train_loss:0.7042575235697416
epoch:2, train_loss:0.680713798097574
epoch:3, train_loss:0.6907698249882036
epoch:4, train_loss:0.6511370800231975
Training completed in 112.12069201469421s


In [109]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk8_softmax')

In [112]:
k=3

In [110]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6690012368583796

In [113]:
# adv training
feature_train2 = feature_train[Ids_train.length2>1]
predicts2 = loop_distance(feature_train2,feature_train2,distanceFun_np,aggFun,returnValue=False,k=k)
distDict = {i:[j for j in item.tolist() if i!=j] for i,item in enumerate(predicts2)}
batch_sampler = AdvSample(distDict,numBatch,batchSize,maxImg)

gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
train_dl= DataLoader(gen_train,num_workers=3,batch_sampler=batch_sampler)

In [114]:
loss_func = loss_func_generator_softmax(batchSize,distanceFun)

In [115]:
model = fit(5, model, loss_func, opt, train_dl, lossBest=None)

epoch:0, train_loss:1.6588831537408255
epoch:1, train_loss:1.5427104221667096
epoch:2, train_loss:1.4602028242225855
epoch:3, train_loss:1.4721440682645703
epoch:4, train_loss:1.4851462140760787
Training completed in 110.97497200965881s


In [116]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk8_softmax')

In [117]:
# validation MAP
pred_train_gen = PredictGenerator2(Ids_train.Imgs.tolist(),transform_test,TTASize, 4,True)
pred_val_gen = PredictGenerator2(Ids_val.Imgs.tolist(),transform_test,TTASize, 2,True)

feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

feature_train = feature_train.reshape(Ids_train.shape[0],TTASize,outDim)
feature_val = feature_val.reshape(Ids_val.shape[0],TTASize,outDim)

predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

MAP(labels,predicts)

0.6661564625850339